In [1]:
# Assignment-4 

In [10]:
##1
#Import Python Libraries.
import numpy as np
import pandas as pd

#Function to read a counts matrix.
def read_counts_matrix(file_path):

    counts_matrix = []  
    with open(file_path, 'r') as file: 
        for line in file:  # Iterate over each line in the file.
            # Extract counts from the line, skipping the base letter and '|' separator.
            parts = line.strip().split()[2:]  
            counts = [int(count) for count in parts]  # Convert count strings to integers.
            counts_matrix.append(counts) 
    return np.array(counts_matrix)  # Convert the counts matrix list to a numpy array and return it.

def compute_frequency_matrix_with_pseudocounts(counts_matrix):
    
    adjusted_counts_matrix = counts_matrix + 1
    total_counts_per_position = np.sum(adjusted_counts_matrix, axis=0)  # Sum counts across each column (position).
    frequency_matrix = adjusted_counts_matrix / total_counts_per_position  # Normalize counts to frequencies.
    return frequency_matrix

def compute_weight_matrix(frequency_matrix):
    background_frequency = 0.25  
    weight_matrix = np.log2(frequency_matrix / background_frequency)  # Compute log2 of frequency ratios.
    return weight_matrix

if __name__ == "__main__":
    
    file_path = 'argR-counts-matrix .txt'  
    counts_matrix = read_counts_matrix(file_path)  # Read the counts matrix from the file.
    frequency_matrix = compute_frequency_matrix_with_pseudocounts(counts_matrix)  # Compute frequency matrix with pseudocounts.
    weight_matrix = compute_weight_matrix(frequency_matrix)  # Compute the weight matrix from the frequency matrix.
    
    # Print the frequency and weight matrices.
    print("Frequency Matrix with Pseudocounts:\n", frequency_matrix)
    print("\nPosition Weight Matrix:\n", weight_matrix)


Frequency Matrix with Pseudocounts:
 [[0.29032258 0.41935484 0.70967742 0.32258065 0.16129032 0.09677419
  0.70967742 0.70967742 0.12903226 0.35483871 0.29032258 0.19354839
  0.25806452 0.83870968 0.16129032 0.09677419 0.09677419 0.83870968]
 [0.25806452 0.16129032 0.06451613 0.22580645 0.09677419 0.12903226
  0.12903226 0.12903226 0.06451613 0.03225806 0.09677419 0.03225806
  0.25806452 0.03225806 0.12903226 0.12903226 0.80645161 0.03225806]
 [0.12903226 0.09677419 0.06451613 0.29032258 0.09677419 0.70967742
  0.09677419 0.09677419 0.03225806 0.06451613 0.03225806 0.06451613
  0.03225806 0.06451613 0.03225806 0.51612903 0.03225806 0.09677419]
 [0.32258065 0.32258065 0.16129032 0.16129032 0.64516129 0.06451613
  0.06451613 0.06451613 0.77419355 0.5483871  0.58064516 0.70967742
  0.4516129  0.06451613 0.67741935 0.25806452 0.06451613 0.03225806]]

Position Weight Matrix:
 [[ 0.21572869  0.74624341  1.50523531  0.36773178 -0.63226822 -1.36923381
   1.50523531  1.50523531 -0.95419631  0.5

In [9]:
##2

#Function to read gene sequences.
def read_sequences(file_path):

    sequences = {} 
    with open(file_path, 'r') as file: 
        for line in file:  # Iterate over each line in the file.
            line = line.strip()  
            if not line:
                continue  
            parts = line.split()  
            gene_id = parts[0]
            sequence = ''.join(parts[1:])
            sequences[gene_id] = sequence  # Store the sequence in the dictionary with the gene ID as the key.
    return sequences  # Return the dictionary of sequences.

#Function to compute the score of a DNA segment using the weight matrix.
def score_segment(segment, weight_matrix):

    base_to_row = {'A': 0, 'C': 1, 'G': 2, 'T': 3}  # Map each base to its corresponding row in the weight matrix.
    score = 0  
    for i, base in enumerate(segment.upper()):  
        if base in base_to_row:  
            score += weight_matrix[base_to_row[base], i]  # Add the score from the weight matrix for this base and position.
    return score  # Return the total score for the segment.

# Function to Scan all sequences with the weight matrix to find the top scoring segments.
def scan_sequences(sequences, weight_matrix, top_n=30):

    top_scores = []  
    for gene_id, sequence in sequences.items():  # Iterate over each gene ID and sequence in the dictionary.
        for i in range(len(sequence) - len(weight_matrix[0]) + 1): 
            segment = sequence[i:i+len(weight_matrix[0])]  
            score = score_segment(segment, weight_matrix)  # Compute the score of the segment using the weight matrix.
            top_scores.append((gene_id, segment, score))  

    top_scores.sort(key=lambda x: x[2], reverse=True)  # Sort the list of scores in descending order based on the score.
    return top_scores[:top_n] 

if __name__ == "__main__":

    sequences = read_sequences('E_coli_K12_MG1655.400_50')
    top_scoring_segments = scan_sequences(sequences, weight_matrix)  # Scan the sequences with the weight matrix to find top scoring segments.

    print("Top 30 scoring segments:")  # Print the top 30 scoring segments.
    for gene_id, segment, score in top_scoring_segments:  # Iterate over each top-scoring segment.
        print(f"Gene ID: {gene_id}, Score: {score}")  # Print the gene ID, sequence segment, and its score.


Top 30 scoring segments:
Gene ID: b3171, Score: 20.981522027249504
Gene ID: 16128258, Score: 20.76324571438529
Gene ID: 16132076, Score: 19.915248807830338
Gene ID: 16131126, Score: 19.093247109808335
Gene ID: 16131238, Score: 18.884660487996918
Gene ID: 16129301, Score: 18.32374346124241
Gene ID: 16132076, Score: 18.117292583774987
Gene ID: 16128684, Score: 18.086704263941563
Gene ID: 16130583, Score: 17.884660487996918
Gene ID: 16131795, Score: 17.884660487996918
Gene ID: 16128026, Score: 17.180116371523088
Gene ID: 16129126, Score: 17.174167105191902
Gene ID: 49176132, Score: 17.034236844047427
Gene ID: 16132077, Score: 16.81427116010552
Gene ID: 16128462, Score: 16.635140888637608
Gene ID: 16131312, Score: 16.392807391667244
Gene ID: b4451, Score: 16.392807391667244
Gene ID: 16128105, Score: 16.303814095747995
Gene ID: 16128258, Score: 16.117292583774987
Gene ID: 16131247, Score: 16.00181536635505
Gene ID: 16131392, Score: 15.934701170496513
Gene ID: 16131884, Score: 15.93470117049